**Предсказание рейтинга ресторанов в TripAdvisor******

Подробнее по признакам:
 City город
 Cuisine Style  кухня
 Ranking  ранг ресторана относительно ресторанов этого города
 Price Range  цены в ресторанах в трех категориях
 Number of Reviews  количество отзывов
 Reviews  2 последних отзыва и их даты
 URL_TA страница ресторана на TripAdvisor
 ID_TA  id ресторана на TripAdvisor
 Rating  рейтинг ресторана


In [ ]:
# Загрузим необходимые библиотеки

import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Зафиксируем RANDOM_SEED         
RANDOM_SEED=1

# Зафиксируем версию пакетов чтобы эксперимент был воспроизводим
!pip freeze > requirements.txt

# Загрузим данные

DATA_DIR='/kaggle/input/sf-dst-restaurant-rating/'
df_train=pd.read_csv(DATA_DIR+'main_task.csv')
df_test=pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission=pd.read_csv(DATA_DIR+'sample_submission.csv')

#объединяем трейн и тест в один датасет
df_train['sample']=1 # помечаем где у нас трейн
df_test['sample']=0 # помечаем где у нас тест
df_test['Rating']=0 # В тесте у нас нет значения Rating поэтому пока просто записываем
data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

# Поменяем немного названия колонок
data.columns=['Restaurant_id', 'City', 'Cuisine_Style', 'Ranking', 'Price_Range', 'Number_of_Reviews', 'Reviews', 'URL_TA', 'ID_TA', 'sample', 'Rating']

# Создадим столбец с данными о наличии пропусков
data['Number_of_Reviews_isNAN']= pd.isna(data['Number_of_Reviews']).astype('uint8')

# Небольшая функция для очистки колонки Cuisine_Style

def clean_name(str_val):
    if pd.isna(str_val): return ["Other"]
    str_val = str_val.strip('[]') 
    str_val = str_val.replace("\'",'') 
    str_val = str_val.strip('""')
    str_val = str_val.split(", ")
    return str_val

data["Cuisine_Style"] = data["Cuisine_Style"].apply(clean_name)

# Создадим колонку с количеством кухонь для каждой строки датасета

data['Num_Cuisine']=data.Cuisine_Style.apply(lambda x: 1 if len(x)==0 else len(x))

# Возьмем среднее значение между $$ и $$$ равное 2.5 и заменим значки в колонке Price_Range значениями
price_range_val={'$': 1, '$$ - $$$': 2.5, '$$$$': 4}
data.Price_Range=data.Price_Range.apply(lambda x: price_range_val[x] if not pd.isnull(x) else x)

# Там где значений нет заполним средним значением Price_Range.mean()
data.Price_Range=data.Price_Range.fillna(round(data.Price_Range.mean(), 1)) #заполнение пропусков средним значением

# Оставим только числовые значения в колонке Restaurant_id
new_id=[]
for num in data['Restaurant_id']:
    new_id.append(int(num[3:]))
data.Restaurant_id=new_id

data['Number_of_Reviews']=data.Number_of_Reviews.fillna(0)  # заполним нулями отсутствие отзывов


# Уберем скобки, кавычки и запятые из колонки Cuisine_Style (на всякий случай)
data.Cuisine_Style = data.Cuisine_Style.apply(lambda x: str(x).strip('[]').strip())
data.Cuisine_Style=data.Cuisine_Style.apply(lambda x:str(x).replace("''", '') if not pd.isnull(x) else x)
data.Cuisine_Style=data.Cuisine_Style.apply(lambda x:str(x).replace("\'", '') if not pd.isnull(x) else x)



In [ ]:
data.head(3)

In [ ]:
# Посмотрим распределение признаков

plt.rcParams['figure.figsize']=(10,7)
df_train['Ranking'].hist(bins=100)

Много ресторанов которые не дотягивают 2500 места в своем городе
Посмотрим что по городам

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

In [ ]:
# Самое большое в Лондоне
# Посмотрим распределение в большом городе
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# Посмотрим на топ 10 городов

for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()    

Получается что Ranking имеет нормальное распределение ,просто в больших городах
больше ресторанов , из за этого мы и имеем смещение

Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] <4].hist(bins=100)

Корреляция признаков

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr())

Создадим список самых топовых кухонь, но для этого придется воспользоваться explode().
Перед этим создадим копию датасета.
Найдем перечень уникальных названий кухонь и оставим 25% от перечня(первые по количеству)
Составим список топовых кухонь

In [ ]:
data_2=data.copy()

data_2.Cuisine_Style=data_2.Cuisine_Style.str.split(',')  
data_2=data_2.explode('Cuisine_Style') 
data_2.Cuisine_Style = data_2.Cuisine_Style.apply(lambda x: str(x).strip('""').strip())
cuisine_list=np.sort(data_2['Cuisine_Style'].unique())[: -1]

cuisine_with_freqs = list(data_2.Cuisine_Style.value_counts())
top_cuisine_count = int(np.percentile(cuisine_with_freqs, 25))
top_cuisine_count

all_cuisine = data_2.Cuisine_Style.value_counts().index
top_cuisine = list(all_cuisine)[:top_cuisine_count]


Псмотрим какие кухни входят в топ

In [ ]:
display(top_cuisine)

In [ ]:
data_3=data.copy() # Сделаем на всякий случай копию датасета

# Еще раз пройдемся по колонке Cuisine_Style

data_3.Cuisine_Style=data_3.Cuisine_Style.str.split(',') 
data_3.Cuisine_Style = data_3.Cuisine_Style.apply(lambda x: str(x).strip('""').strip())
data_3.Cuisine_Style = data_3.Cuisine_Style.apply(lambda x: str(x).strip('[]').strip())
data_3.Cuisine_Style=data_3.Cuisine_Style.apply(lambda x:str(x).replace("\'", '') if not pd.isnull(x) else x)

# Возьмем первые три топовых кухни и создадим для каждой из них колонку в датасете
counter_1=[]
counter_2=[]
counter_3=[]
for i in range(0, len(data)):
    cuisine_val=data_3['Cuisine_Style'][i].count(top_cuisine[0])
    counter_1.append(cuisine_val)
    cuisine_val_2=data_3['Cuisine_Style'][i].count(top_cuisine[1])
    counter_2.append(cuisine_val_2)
    cuisine_val_3=data_3['Cuisine_Style'][i].count(top_cuisine[2])
    counter_3.append(cuisine_val_3)
    
data_3['Top_Cuisine'] = counter_1
data_3['Top_Cuisine_2'] = counter_2
data_3['Top_Cuisine_3'] = counter_3

# Добавим еще признаки применив логарифмирование и квадратный корень для колонок Ranking и Restaurant_id
counter_4=[]
for i in range(0, len(data)):
    ranking_val=np.log(data_3.Ranking[i])
    counter_4.append(ranking_val)
data_3['Ranking_log'] = counter_4

counter_5=[]
for i in range(0, len(data)):
    id_val=np.sqrt(data_3.Restaurant_id[i])
    counter_5.append(id_val)
data_3['Restaurant_id_sqrt'] = counter_5    

# Добавим признаки по городам применив get_dummies

data_4=pd.get_dummies(data_3, columns=['City',], dummy_na=True)

In [ ]:
# Удалим ненужные колонки 

data_4=data_4.drop(['Cuisine_Style', 'Reviews', 'URL_TA', 'ID_TA', 'City_nan'], axis=1)

In [ ]:
# Посмотрим что получилось
data_4.head(5)

In [ ]:
data_4.info()

Проверяем что получилось

In [ ]:
df_preproc = data_4
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data=df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data=df_preproc.query('sample == 0').drop(['sample'], axis=1)

y=train_data.Rating.values   # наш таргет
X=train_data.drop(['Rating'], axis=1)

In [ ]:
# Перед отправкой наших данных на обучение ,разделим данные на еще один тест и трейн, для валидации
# это поможет проверить работу модели ло отправки Submission на kaggle

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Выделим 20 процентов на валидацию
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)


In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Модель

In [ ]:
# Импортируем необходимые библиотеки
from sklearn.ensemble import RandomForestRegressor #инструмент для создания и обучения модели
from sklearn import metrics #инсструмент для оценки точности модели

In [ ]:
# Создаем модель
model=RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# исспользуем обученную модель для предсказания рейтинга рессторанов в тестовой выборке
# предсказанные значения записываем в переменную y_pred
y_pred=model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения(y_pred) с реальными(y_test), и ссмотрим на сколько они  в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# Выведем важные признаки для модели
plt.rcParams['figure.figsize']= (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
# Submission  
# Если все устраивает готовим submission на kaggle

In [ ]:
test_data.sample(10)

In [ ]:
test_data= test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)